In [8]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly_dataframe import plot, plot_comparison

## Data sources

In [9]:
PATH_TO_DATA_FOLDER = "../"

In [10]:
# Load the dataset.
df_smooth = pd.read_csv(PATH_TO_DATA_FOLDER + "dataframe_smooth.csv", header = [0, 1], index_col = 0)
df_smooth.index.name = "Datetime"
df_smooth.index = pd.to_datetime(df_smooth.index)
freq = "D"
df_smooth.index.freq = freq
df_smooth.dropna(inplace = True)
df_smooth

AdminStrata                         Abyan                     \
Indicator   3 Months Anomaly (%) Rainfall Cereals and tubers   
Datetime                                                       
2018-08-22                      94.347604           0.329240   
2018-08-23                      93.856583           0.330338   
2018-08-24                      93.468502           0.331272   
2018-08-25                      93.146923           0.332116   
2018-08-26                      92.918664           0.332991   
...                                   ...                ...   
2020-03-27                     131.160981           0.493156   
2020-03-28                     132.376130           0.494493   
2020-03-29                     133.499938           0.495830   
2020-03-30                     134.618325           0.497167   
2020-03-31                     135.935213           0.498504   

AdminStrata                                                             \
Indicator   Exchange rate (unofficial) (USD/LCU)        FCS   Fatality   
Datetime                                                                 
2018-08-22                              0.371628  33.261476   9.188630   
2018-08-23                              0.373522  33.224058   9.534132   
2018-08-24                              0.374778  33.146743  10.091062   
2018-08-25                              0.375686  33.117270  10.885305   
2018-08-26                              0.376715  33.536064  11.862721   
...                                          ...        ...        ...   
2020-03-27                              0.545576  36.145245  13.782164   
2020-03-28                              0.545650  36.555222  12.569469   
2020-03-29                              0.545725  37.031010  11.160477   
2020-03-30                              0.545799  37.643389  10.656717   
2020-03-31                              0.545873  38.515442  12.871263   

AdminStrata                                                          ...  \
Indicator          Lat        Lon      NDVI NDVI Anomaly Population  ...   
Datetime                                                             ...   
2018-08-22   13.704878  46.158142  0.098887    96.886320   583193.0  ...   
2018-08-23   13.704878  46.158142  0.098957    96.789475   583193.0  ...   
2018-08-24   13.704878  46.158142  0.099032    96.694350   583193.0  ...   
2018-08-25   13.704878  46.158142  0.099113    96.600567   583193.0  ...   
2018-08-26   13.704878  46.158142  0.099198    96.504903   583193.0  ...   
...                ...        ...       ...          ...        ...  ...   
2020-03-27   13.704878  46.158142  0.117874   108.349542   583193.0  ...   
2020-03-28   13.704878  46.158142  0.117605   108.541215   583193.0  ...   
2020-03-29   13.704878  46.158142  0.117348   108.721413   583193.0  ...   
2020-03-30   13.704878  46.158142  0.117099   108.880750   583193.0  ...   
2020-03-31   13.704878  46.158142  0.116851   109.010760   583193.0  ...   

AdminStrata      Taizz                                              \
Indicator          FCS    Fatality        Lat        Lon      NDVI   
Datetime                                                             
2018-08-22   38.654368  159.378381  13.416517  43.778161  0.198396   
2018-08-23   38.191166  154.755214  13.416517  43.778161  0.198518   
2018-08-24   37.869192  154.069511  13.416517  43.778161  0.198656   
2018-08-25   37.465880  156.254554  13.416517  43.778161  0.198812   
2018-08-26   37.068649  160.506003  13.416517  43.778161  0.198986   
...                ...         ...        ...        ...       ...   
2020-03-27   37.061938   59.875554  13.416517  43.778161  0.161132   
2020-03-28   36.888902   60.220486  13.416517  43.778161  0.160547   
2020-03-29   36.942218   62.104671  13.416517  43.778161  0.159994   
2020-03-30   37.354415   63.952611  13.416517  43.778161  0.159465   
2020-03-31   38.317418   62.274508  13.416517  43.778161  0.158944   

AdminStrata         

In [11]:
PROVINCES = df_smooth.columns.levels[0].unique()
PROVINCES

Index(['Abyan', 'Aden', 'Al Bayda', 'Al Dhale'e', 'Al Hudaydah', 'Al Jawf',
       'Al Maharah', 'Al Mahwit', 'Amanat Al Asimah', 'Amran', 'Dhamar',
       'Hadramaut', 'Hajjah', 'Ibb', 'Lahj', 'Marib', 'Raymah', 'Sa'ada',
       'Sana'a', 'Shabwah', 'Taizz'],
      dtype='object', name='AdminStrata')

In [12]:
PREDICTORS = df_smooth.columns.levels[1].unique()
PREDICTORS

Index(['3 Months Anomaly (%) Rainfall', 'Cereals and tubers',
       'Exchange rate (unofficial) (USD/LCU)', 'FCS', 'Fatality', 'Lat', 'Lon',
       'NDVI', 'NDVI Anomaly', 'Population', 'Rainfall (mm)', 'Ramadan',
       'rCSI'],
      dtype='object', name='Indicator')

In [13]:
import STE_fast

# Lag importance

In [14]:
# Select only the endogenous indicators (FCS indicator).
select = df_smooth.columns.get_level_values(1).isin(["FCS"])
df_fcs = df_smooth.loc[:, select]
df_fcs.head()

AdminStrata,Abyan,Aden,Al Bayda,Al Dhale'e,Al Hudaydah,Al Jawf,Al Maharah,Al Mahwit,Amanat Al Asimah,Amran,...,Hadramaut,Hajjah,Ibb,Lahj,Marib,Raymah,Sa'ada,Sana'a,Shabwah,Taizz
Indicator,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,...,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS
Datetime,,,,,,,,,,,,,,,,,,,,,
2018-08-22,33.261476,20.346337,49.713942,38.178731,18.468202,35.595758,16.810107,25.198105,24.237758,36.955111,...,18.426048,32.832390,40.971215,49.293182,48.133102,50.058556,30.168480,31.764405,30.585680,38.654368
2018-08-23,33.224058,21.495059,49.463745,38.957649,18.776979,35.448330,16.696007,25.010288,24.155927,38.077905,...,19.454945,32.757887,40.354229,48.501429,47.204925,49.841190,29.681345,32.117659,30.001003,38.191166
2018-08-24,33.146743,22.841084,49.059963,39.610140,19.135952,35.323247,16.725691,24.958883,24.154543,39.261164,...,20.827429,32.674261,39.630501,47.542702,46.217404,49.662240,28.880328,32.422062,29.425419,37.869192
2018-08-25,33.117270,23.909307,48.214405,39.957692,19.461947,35.420266,16.844693,25.008405,24.206217,40.559018,...,22.276649,32.542064,38.946643,46.647968,45.176024,49.807669,27.922647,32.661162,28.964254,37.465880
2018-08-26,33.536064,24.930476,47.461153,40.246702,19.858203,35.807343,17.059417,25.134007,24.279413,42.054653,...,23.742988,32.242924,38.179497,46.052743,44.205951,50.052188,26.818249,32.781012,28.751100,37.068649


In [15]:
from tqdm import tqdm

lags = 20
def history_length_Y(serie):
    adminstrata = serie.name[0]
    y = list()
    for k in range(lags):
        result = STE_fast.entropy_rate(serie, m = 3, k = k+1)
        y.append(result)
        
    return y  

tqdm.pandas()
df_results_Y = df_fcs.progress_apply(history_length_Y)
df_results_Y.index = df_results_Y.index  + 1
df_results_Y.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:20<00:00,  1.05it/s]


AdminStrata,Abyan,Aden,Al Bayda,Al Dhale'e,Al Hudaydah,Al Jawf,Al Maharah,Al Mahwit,Amanat Al Asimah,Amran,...,Hadramaut,Hajjah,Ibb,Lahj,Marib,Raymah,Sa'ada,Sana'a,Shabwah,Taizz
Indicator,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,...,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS,FCS
1,0.566969,0.631538,0.584318,0.655529,0.572300,0.602611,0.624524,0.723790,0.599165,0.607110,...,0.541265,0.606988,0.582314,0.675026,0.598436,0.646204,0.582113,0.600565,0.628577,0.603047
2,0.537445,0.611454,0.552837,0.608087,0.549964,0.571904,0.602531,0.686839,0.563566,0.591954,...,0.511772,0.593044,0.557452,0.632596,0.581149,0.600310,0.557204,0.582830,0.611848,0.580558
3,0.477926,0.573697,0.532715,0.568060,0.525443,0.543415,0.572515,0.640104,0.543142,0.549474,...,0.500767,0.563540,0.529441,0.604428,0.550186,0.569662,0.548508,0.567076,0.573342,0.529661
4,0.461742,0.544294,0.522441,0.548453,0.489862,0.522842,0.533974,0.596918,0.527846,0.518352,...,0.488128,0.548001,0.494908,0.558715,0.533346,0.542350,0.522352,0.552786,0.538499,0.480722
5,0.445376,0.527866,0.500165,0.528654,0.475301,0.507278,0.499344,0.569285,0.504615,0.504038,...,0.478698,0.526642,0.481261,0.545091,0.521550,0.526440,0.508320,0.518495,0.524239,0.463535


In [16]:
import seaborn as sns
from ipywidgets import interact, widgets, fixed

def plot_df(name, df):
    group = df[name]

    # Set default trace colors with colorway.
    colorway = sns.color_palette("hls", 8).as_hex()
    layout = go.Layout(colorway = colorway)

    fig = go.Figure(layout = layout)

    for column in group.columns:
        fig.add_trace(go.Scatter(x = group.index, y = group[column], name = column, mode = "lines", 
                                 showlegend = True, line = dict(width = 1.5)))

    # Edit the layout.
    fig.update_layout(title = dict(text = name, y = 0.9, x = 0.5))
    fig.update_layout(xaxis_title = dict(text = "Lags"))

    fig.show()

In [17]:
# Create figure.
w = widgets.ToggleButtons(options = df_results_Y.columns.levels[0], description = "Adminstrata:", 
                          disabled = False)
p = interact(plot_df, name = w, df = fixed(df_results_Y))

interactive(children=(ToggleButtons(description='Adminstrata:', options=('Abyan', 'Aden', 'Al Bayda', "Al Dhal…

In [20]:
# Select only the exogenous indicators.
df_no_fcs = df_smooth.loc[:, ~select]
# Delete static features.
df_no_fcs = df_no_fcs.drop(columns = ["Lat", "Lon", "Population"], axis = 1, level = 1)
df_no_fcs.head()

AdminStrata                         Abyan                     \
Indicator   3 Months Anomaly (%) Rainfall Cereals and tubers   
Datetime                                                       
2018-08-22                      94.347604           0.329240   
2018-08-23                      93.856583           0.330338   
2018-08-24                      93.468502           0.331272   
2018-08-25                      93.146923           0.332116   
2018-08-26                      92.918664           0.332991   

AdminStrata                                                            \
Indicator   Exchange rate (unofficial) (USD/LCU)   Fatality      NDVI   
Datetime                                                                
2018-08-22                              0.371628   9.188630  0.098887   
2018-08-23                              0.373522   9.534132  0.098957   
2018-08-24                              0.374778  10.091062  0.099032   
2018-08-25                              0.375686  10.885305  0.099113   
2018-08-26                              0.376715  11.862721  0.099198   

AdminStrata                                                \
Indicator   NDVI Anomaly Rainfall (mm) Ramadan       rCSI   
Datetime                                                    
2018-08-22     96.886320     12.905566       0  38.673283   
2018-08-23     96.789475     13.055369       0  36.022125   
2018-08-24     96.694350     13.230741       0  35.751261   
2018-08-25     96.600567     13.421418       0  36.892743   
2018-08-26     96.504903     13.607525       0  38.731071   

AdminStrata                          Aden  ...    Shabwah  \
Indicator   3 Months Anomaly (%) Rainfall  ...       rCSI   
Datetime                                   ...              
2018-08-22                     119.834645  ...  35.479993   
2018-08-23                     118.992363  ...  34.813709   
2018-08-24                     118.437081  ...  34.936023   
2018-08-25                     118.068028  ...  35.564732   
2018-08-26                     117.949785  ...  36.492644   

AdminStrata                         Taizz                     \
Indicator   3 Months Anomaly (%) Rainfall Cereals and tubers   
Datetime                                                       
2018-08-22                     112.979629           0.349572   
2018-08-23                     112.358334           0.349713   
2018-08-24                     112.047979           0.349767   
2018-08-25                     111.937911           0.349774   
2018-08-26                     112.092116           0.349796   

AdminStrata                                                             \
Indicator   Exchange rate (unofficial) (USD/LCU)    Fatality      NDVI   
Datetime                                                                 
2018-08-22                              0.370249  159.378381  0.198396   
2018-08-23                              0.372977  154.755214  0.198518   
2018-08-24                              0.375668  154.069511  0.198656   
2018-08-25                              0.378338  156.254554  0.198812   
2018-08-26                              0.381016  160.506003  0.198986   

AdminStrata                                                
Indicator   NDVI Anomaly Rainfall (mm) Ramadan       rCSI  
Datetime                                                   
2018-08-22    101.483325     41.573789       0  50.880803  
2018-08-23    101.253732     42.602619       0  50.207440  
2018-08-24    101.018612     43.672707       0  50.045504  
2018-08-25    100.780032     44.768737       0  50.074108  
2018-08-26    100.540305     45.821729       0  50.120693  

[5 rows x 189 columns]

In [21]:
lags = 180
def history_length_X(serie):
    adminstrata = serie.name[0]
    y = list()
    for k in range(lags):
        result = STE_fast.calc_ste(serie, df_fcs[adminstrata]["FCS"], m = 3, kx = k+1, ky = 1)
        y.append(result)        
    return y  

tqdm.pandas()
df_results_X = df_no_fcs.progress_apply(history_length_X)
df_results_X.index = df_results_X.index  + 1
df_results_X.head()

100%|██████████████████████████████████████████████████████████████████████████████| 189/189 [1:05:59<00:00, 20.95s/it]


AdminStrata                         Abyan                     \
Indicator   3 Months Anomaly (%) Rainfall Cereals and tubers   
1                                0.037594           0.043091   
2                                0.051777           0.047408   
3                                0.059038           0.048854   
4                                0.067123           0.051299   
5                                0.078083           0.055757   

AdminStrata                                                           \
Indicator   Exchange rate (unofficial) (USD/LCU)  Fatality      NDVI   
1                                       0.041271  0.045860  0.012596   
2                                       0.063924  0.072590  0.021600   
3                                       0.079130  0.108709  0.025323   
4                                       0.086881  0.152182  0.033630   
5                                       0.088183  0.198971  0.037446   

AdminStrata                                                 \
Indicator   NDVI Anomaly Rainfall (mm)   Ramadan      rCSI   
1               0.018639      0.038690  0.000515  0.037028   
2               0.020374      0.053329  0.000780  0.056397   
3               0.020999      0.071488  0.001080  0.082088   
4               0.031491      0.092393  0.001343  0.119159   
5               0.036449      0.100635  0.001602  0.143263   

AdminStrata                          Aden  ...   Shabwah  \
Indicator   3 Months Anomaly (%) Rainfall  ...      rCSI   
1                                0.016687  ...  0.040062   
2                                0.023342  ...  0.067258   
3                                0.033289  ...  0.100345   
4                                0.042465  ...  0.139134   
5                                0.057177  ...  0.166957   

AdminStrata                         Taizz                     \
Indicator   3 Months Anomaly (%) Rainfall Cereals and tubers   
1                                0.032051           0.029807   
2                                0.045723           0.034597   
3                                0.062087           0.041534   
4                                0.079527           0.054632   
5                                0.100019           0.072187   

AdminStrata                                                           \
Indicator   Exchange rate (unofficial) (USD/LCU)  Fatality      NDVI   
1                                       0.013972  0.058035  0.012896   
2                                       0.024188  0.081592  0.019440   
3                                       0.031380  0.092703  0.028196   
4                                       0.048994  0.110972  0.038691   
5                                       0.054479  0.131541  0.042846   

AdminStrata                                                 
Indicator   NDVI Anomaly Rainfall (mm)   Ramadan      rCSI  
1               0.022972      0.030864  0.000541  0.053644  
2               0.028235      0.051365  0.000818  0.092200  
3               0.045686      0.078134  0.001101  0.131380  
4               0.052050      0.094531  0.001389  0.175146  
5               0.059124      0.110630  0.001683  0.235079  

[5 rows x 189 columns]

In [22]:
# Create figure.
w = widgets.ToggleButtons(options = df_results_X.columns.levels[0], description = "Adminstrata:", 
                          disabled = False)
p = interact(plot_df, name = w, df = fixed(df_results_X))

interactive(children=(ToggleButtons(description='Adminstrata:', options=('Abyan', 'Aden', 'Al Bayda', "Al Dhal…

In [23]:
def quantile_25(x):
    return x.quantile(q = 0.25)

def quantile_75(x):
    return x.quantile(q = 0.75)

In [24]:
# Create figure.
fig = go.Figure()

def plot_quantiles(group):
    statistics_group = group.agg([np.mean, quantile_25, quantile_75], axis = "columns")
    
    # Random color.
    color = list(np.random.choice(range(256), size = 3))
    color_mean = "rgb" + str(tuple(color))
    color.append(0.3)
    color_quantile = "rgba" + str(tuple(color))

    # Plot mean.
    fig.add_trace(go.Scatter(x = statistics_group.index, y = statistics_group["mean"], mode = "lines", name = group.name, legendgroup = group.name, 
                             line = dict(width = 1.5, color = color_mean)))
    # Plot quantiles.
    fig.add_trace(go.Scatter(x = statistics_group.index, y = statistics_group["quantile_25"], legendgroup = group.name, showlegend = False, fill = None, mode = "lines", fillcolor = color_quantile, line = dict(width = .2, color = color_quantile)))
    fig.add_trace(go.Scatter(x = statistics_group.index, y = statistics_group["quantile_75"], legendgroup = group.name, showlegend = False, fill = "tonexty", mode = "lines", fillcolor = color_quantile, line = dict(width = .2, color = color_quantile)))

    fig.update_layout(xaxis_title = dict(text = "Lags", font = dict(size = 15)), 
                      yaxis_title = dict(text = "Symbolic Transfer Entropy", font = dict(size = 15)))
    
df_results_X.groupby(axis = 1, level = 1).apply(plot_quantiles);
fig.show()

# Feature importance

In [27]:
df_feature_importance = df_smooth.copy()
# Delete static features.
df_feature_importance = df_feature_importance.drop(columns = ["Lat", "Lon", "Population"], axis = 1, level = 1)
df_feature_importance.head()

AdminStrata                         Abyan                     \
Indicator   3 Months Anomaly (%) Rainfall Cereals and tubers   
Datetime                                                       
2018-08-22                      94.347604           0.329240   
2018-08-23                      93.856583           0.330338   
2018-08-24                      93.468502           0.331272   
2018-08-25                      93.146923           0.332116   
2018-08-26                      92.918664           0.332991   

AdminStrata                                                             \
Indicator   Exchange rate (unofficial) (USD/LCU)        FCS   Fatality   
Datetime                                                                 
2018-08-22                              0.371628  33.261476   9.188630   
2018-08-23                              0.373522  33.224058   9.534132   
2018-08-24                              0.374778  33.146743  10.091062   
2018-08-25                              0.375686  33.117270  10.885305   
2018-08-26                              0.376715  33.536064  11.862721   

AdminStrata                                                          ...  \
Indicator        NDVI NDVI Anomaly Rainfall (mm) Ramadan       rCSI  ...   
Datetime                                                             ...   
2018-08-22   0.098887    96.886320     12.905566       0  38.673283  ...   
2018-08-23   0.098957    96.789475     13.055369       0  36.022125  ...   
2018-08-24   0.099032    96.694350     13.230741       0  35.751261  ...   
2018-08-25   0.099113    96.600567     13.421418       0  36.892743  ...   
2018-08-26   0.099198    96.504903     13.607525       0  38.731071  ...   

AdminStrata                         Taizz                     \
Indicator   3 Months Anomaly (%) Rainfall Cereals and tubers   
Datetime                                                       
2018-08-22                     112.979629           0.349572   
2018-08-23                     112.358334           0.349713   
2018-08-24                     112.047979           0.349767   
2018-08-25                     111.937911           0.349774   
2018-08-26                     112.092116           0.349796   

AdminStrata                                                              \
Indicator   Exchange rate (unofficial) (USD/LCU)        FCS    Fatality   
Datetime                                                                  
2018-08-22                              0.370249  38.654368  159.378381   
2018-08-23                              0.372977  38.191166  154.755214   
2018-08-24                              0.375668  37.869192  154.069511   
2018-08-25                              0.378338  37.465880  156.254554   
2018-08-26                              0.381016  37.068649  160.506003   

AdminStrata                                                          
Indicator        NDVI NDVI Anomaly Rainfall (mm) Ramadan       rCSI  
Datetime                                                             
2018-08-22   0.198396   101.483325     41.573789       0  50.880803  
2018-08-23   0.198518   101.253732     42.602619       0  50.207440  
2018-08-24   0.198656   101.018612     43.672707       0  50.045504  
2018-08-25   0.198812   100.780032     44.768737       0  50.074108  
2018-08-26   0.198986   100.540305     45.821729       0  50.120693  

[5 rows x 210 columns]

In [30]:
STE_fast.compute_T?

In [34]:
rows_ste = list()
rows_ste_rank = list()

def feature_importance(group):
    adminstrata = group.name
    T = STE_fast.compute_T(group, m = 3, h = 1, kx = 1, ky = 1)
    # Select row of the FCS.
    row = T.loc[adminstrata].loc["FCS"].drop((adminstrata, "FCS")).droplevel(axis = 0, level = 0)
    row.name = adminstrata + " - FCS"
    rows_ste.append(row)
    rows_ste_rank.append(row.rank(method = "max", ascending = False))

tqdm.pandas()
df_feature_importance.groupby(level = 0, axis = 1).progress_apply(feature_importance);
T_fcs = pd.concat(rows_ste, axis = 1).transpose()
T_fcs_rank = pd.concat(rows_ste_rank, axis = 1).transpose()




  0%|                                                                                           | 0/21 [00:00<?, ?it/s]


 10%|███████▉                                                                           | 2/21 [00:01<00:10,  1.88it/s]


 14%|███████████▊                                                                       | 3/21 [00:02<00:12,  1.39it/s]


 19%|███████████████▊                                                                   | 4/21 [00:03<00:13,  1.22it/s]


 24%|███████████████████▊                                                               | 5/21 [00:04<00:15,  1.02it/s]


 29%|███████████████████████▋                                                           | 6/21 [00:05<00:15,  1.04s/it]


 33%|███████████████████████████▋                                                       | 7/21 [00:07<00:15,  1.10s/it]


 38%|███████████████████████████████▌                                                   | 8/21 [00:08<00:14,  1.13s/it]


 43%|████████████████

In [47]:
fig = go.Figure(data = go.Heatmap(z = T_fcs, x = T_fcs.columns, y = T_fcs.index, xgap = 2, ygap = 2, hoverinfo = "x+y+z", colorbar = {"title": "STE"},
                                  colorscale = "Reds", reversescale = False, hovertemplate = "<br><b>Indicator</b>: %{x}<br>" + 
                                  "<b>Indicator</b>: %{y}" + "<br><b>STE</b>: %{z:.3f}<br>", hoverlabel = dict(namelength = 0)))
fig.update_layout(width = 500, height = 600, yaxis = dict(autorange = "reversed", tickfont = dict(size = 10)), 
                  xaxis = dict(tickfont = dict(size = 10)))
fig.update_layout(title = dict(text = "T - Influence different indicators for FCS"))
fig.show()

In [36]:
fig = go.Figure(data = [go.Box(y = T_fcs_rank[col], name = col) for i, col in enumerate(T_fcs_rank)])
fig.update_layout(yaxis = dict(autorange = "reversed"))
fig.update_layout(yaxis_title = dict(text = "Ranking position importance", font = dict(size = 15)))
fig.show()